# Drone Delivery

### Imports

In [157]:
import math
import numpy as np
import pandas as pd

import psycopg2

import json

import gmaps
import gmaps.geojson_geometries

import math

from geographiclib.geodesic import Geodesic

### Utility Functions

In [150]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

## Set up Postgres DB connection
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

## Connect to Postgres DB
cursor = connection.cursor()

def my_calculate_distance(point_1, point_2):
    "Given two points in (latitude, longitude) format, calculate the distance between them in miles"
    
    geod = Geodesic.WGS84


    g = geod.Inverse(point_1[0], point_1[1], point_2[0], point_2[1])
    miles = g['s12'] / 1000 * 0.621371
    
    return miles

## Import Google Maps API key
f = open('gmap_api_key.txt', 'r')
my_api_key = f.read()
f.close()

gmaps.configure(api_key=my_api_key)

## Business Problem
Acme Gourmet Meals (AGM) is at an important place in the company's history. We are at a crossroads with regards to business expansion and there are a number of options for how to proceed, ranging from more traditional approaches like adding new physical store locations to the futuristic with drone and robot deliveries. When considering the latter options, we must take into account account the maximum range of the delivery along with the population/customer density within that range. Additionally, cost of drones and maintenance will need to be taken into account to determine the overall efficacy of the drone option. We will start by defining the delivery range and potential customer impact of using delivery drones at AGM.

According to the LA-based food delivery company, Flyby, drones can deliver food within a 1.5 mile range and can travel at up-to 40 miles per hour. With this in mind, we will investigate the number of customers within a 1.5 mile radius of the Berkeley AGM store.

In [44]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select count(distinct(cu.customer_id)) as "Unique Customers", count(s.sale_id) "Number of Transactions", sum(s.total_amount) as "Annual Spending ($)"
from customers as cu
     join sales s
         on cu.customer_id = s.customer_id
where cu.closest_store_id = 1
and distance <= 1.5;
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,Unique Customers,Number of Transactions,Annual Spending ($)
0,790,51580,3128808


## Customer History
Focusing on customers within a 1.5 mile radius of the Berkeley AGM store, we can see in 2020 there were 790 customers who spent a total of \$3,128,808 in 51,580 transactions. With this knowledge in mind, we want to project the potential impact of drone delivery on sales and earnings. In the below table, we see the sales amounts based on a potential increase in sales of 2\%, 4\%, and 6\% respectively. It is difficult to gauge the future sales amounts if the company implemented drone delivery but considering the customer benefits of delivery along with the novelty of drone use, we provided a range of possible future sales figures based on existing customer behavior.

In [82]:
rollback_before_flag = True
rollback_after_flag = True

query = """
select *
from
(select 
    1 as Order,
    '2020 Sales' as "Scenario",
    sum(s.total_amount) as "Annual Spending ($)",
    'N/A' as "Sales Increase ($)"
from customers as cu
     join sales s
         on cu.customer_id = s.customer_id
where cu.closest_store_id = 1
and distance <= 1.5
union
select 
    2 as Order,
    '2% Sales Increase' as "Scenario",
    sum(s.total_amount) * 1.02 as "Annual Spending ($)",
    (sum(s.total_amount) * 1.02 - sum(s.total_amount))::varchar as "Sales Increase ($)"
from customers as cu
     join sales s
         on cu.customer_id = s.customer_id
where cu.closest_store_id = 1
and distance <= 1.5
union
select 
    3 as Order,
    '4% Sales Increase' as "Scenario",
    sum(s.total_amount) * 1.04 as "Annual Spending ($)",
    (sum(s.total_amount) * 1.04 - sum(s.total_amount))::varchar as "Sales Increase ($)"
from customers as cu
     join sales s
         on cu.customer_id = s.customer_id
where cu.closest_store_id = 1
and distance <= 1.5
union
select 
    4 as Order,
    '6% Sales Increase' as "Scenario",
    sum(s.total_amount) * 1.06 as "Annual Spending ($)",
    (sum(s.total_amount) * 1.06 - sum(s.total_amount))::varchar as "Sales Increase ($)"
from customers as cu
     join sales s
         on cu.customer_id = s.customer_id
where cu.closest_store_id = 1
and distance <= 1.5) q
order by 1
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)[['Scenario', 'Annual Spending ($)', 'Sales Increase ($)']]
df

,Scenario,Annual Spending ($),Sales Increase ($)
0,2020 Sales,3128808.00,N/A
1,2% Sales Increase,3191384.16,62576.16
2,4% Sales Increase,3253960.32,125152.32
3,6% Sales Increase,3316536.48,187728.48


## Potential Customer Base
While it's important to understand past customer behavior and how that may affect a decision to implement drone delivery, we must also understand the untapped customer base within the delivery range. To do this, we have identified the zip codes within the delivery range of the Berkeley store and analyzed the number of residents in these areas. While we have several hundred existing customers within the drone delivery range, we can see in the charts below that there are over 100,000 residents living within deliverable zip codes. This means the existing customer base is less than one percent of the population which means there is tremendous potential for growth even within the 1.5 mile range reachable by delivery drones.

A small caveat regarding the number of residents is that the distance calculations are based on the center of the delivery zip code rather than individual addresses so there may be some residents in a zip code that are eligible for delivery and some would be just out of range. However, the total number of residents is likely reasonably close to the genuine number because there are also some residents that are within delivery range even though the center of the zip code does not register in our results.

In [135]:
rollback_before_flag = True
rollback_after_flag = True

query = """
select s.latitude as "berkeley_lat", s.longitude as "berkeley_long", zc.*
from stores s, zip_codes zc
where s.city = 'Berkeley'
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

zips_in_range = []
for index, row in df.iterrows():
    berkeley_location = (row['berkeley_lat'], row['berkeley_long'])
    target_location = (row['latitude'], row['longitude'])
    distance = my_calculate_distance(berkeley_location, target_location)
    if distance <= 2:
        zips_in_range.append({**row, 'distance': distance })
        
zips_in_range = pd.DataFrame(data=zips_in_range)
zips_in_range[['city', 'state', 'population', 'distance']]

,city,state,population,distance
0,Emeryville,CA,30289,1.944523
1,Oakland,CA,22811,1.477601
2,Oakland,CA,17041,1.418286
3,Berkeley,CA,17092,1.584381
4,Berkeley,CA,21937,1.013126
5,Berkeley,CA,29190,0.776715
6,Berkeley,CA,13365,1.386033
7,Berkeley,CA,11740,1.679480
8,Berkeley,CA,2971,1.312140


In [136]:
print('Number of customers in deliverable zip codes:', zips_in_range[zips_in_range['distance'] <= 1.5]['population'].sum())

Number of customers in deliverable zip codes: 107315


## Costs
In order to get a full understanding of the potential of drone deliver, we must also consider the costs associated with the drones. Based on literature analyzing the cost of <cite>[drone delivery][1]</cite>, the average cost of a single delivery drone is \\$4,000. For our drone delivery proof-of-concept, we recommend an initial approach using one or two delivery drones in order to better understand their viability and the customer response. The \\$8,000 initial investment pales in comparison to the millions of dollars our customers spend annually who are within our delivery range so we would recommend beginning a proof-of-concept whenever possible.


[1]: https://libjournals.mtsu.edu/index.php/jfee/article/download/1512/1090/4144

## Drone Range - Visualized

In [186]:
berkeley_store = (df.iloc[0]['berkeley_lat'],df.iloc[0]['berkeley_long'])
fig = gmaps.figure(center=berkeley_store, zoom_level=12)
drawing = gmaps.drawing_layer(features=[
    gmaps.Circle(
        radius=2414,  # 1.5 miles in meters
        center=berkeley_store,
        stroke_color='red', fill_color=(255, 0, 132)
    )
], mode='DISABLED')
fig.add_layer(drawing)
marker_layer = gmaps.marker_layer([berkeley_store], info_box_content=['Berkeley AGM Store'])
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))